# VQE Calculation
File for the full ground state calculation of the VQE on the IBM Cloud

In [1]:
import numpy as np
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import NELDER_MEAD, GradientDescent
from qiskit_nature.circuit.library.ansatzes import UCC
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit.utils import algorithm_globals
from qiskit.primitives import Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, Session

from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
import datetime
import pylab

import API.Keys
from qiskit import QuantumCircuit

from src.hamiltonian.FermionicHamiltonian import FermionicHamiltonian
from src.molecule.BeH2 import BeH2

## Creating the BeH2 molecule
The concrete molecule specifics can be found in the BeH2 class file

In [2]:
num_orbitals = 14
num_electrons = 6
molecule = BeH2(num_orbitals, num_electrons)

In [3]:
# create the hamiltonian object and get the operator
hamiltonian = FermionicHamiltonian(molecule)
operator = hamiltonian.get_hamiltonian_op()

## Getting IBM Cloud service

In [4]:
# get API key
key = API.Keys.IBM_BA_KEY
instance = API.Keys.CRN_VALUE
service = QiskitRuntimeService(channel="ibm_cloud", token=key, instance=instance)

In [5]:
algorithm_globals.random_seed = 50
# use ansatz TwoLocal
#ansatz2 = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
counts = []
values = []
params = []

In [6]:
# Define callback routine which will be invoked after every shot
def store_intermediate_result(eval_count, parameters, mean, std):
    time = datetime.datetime.now()
    print(f"Iteration done at {time}")
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)

In [7]:
optimizer = NELDER_MEAD(maxiter=500, maxfev=500)
print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
converter = QubitConverter(mapper=JordanWignerMapper(), two_qubit_reduction=True)
ucc_ansatz = UCC(qubit_converter=converter, num_spin_orbitals=14, num_particles=(0,6), excitations='sd', alpha_spin=True, beta_spin=True, max_spin_excitation=1, generalized=True,preserve_spin=True, reps=5)

Optimizer: NELDER_MEAD        

In [8]:
# create session for the estimator object
with Session(service=service, backend="ibmq_qasm_simulator"):
    estimator = Estimator()
    # Use VQE with estimator, ansatz, optimizer
    vqe = VQE(estimator, ucc_ansatz, optimizer, callback=store_intermediate_result)
    # invoke computation on the operator
    time = datetime.datetime.now()
    print(f"Starting calculation at: {time}")
    result = vqe.compute_minimum_eigenvalue(operator=operator)
    time = datetime.datetime.now()
    print(f"Ended calculation at: {time}")

print("\rOptimization complete");


converge_counts = np.asarray(counts)
converge_vals = np.asarray(values)

Starting calculation at: 2023-04-27 16:14:38.191945
Iteration done at 2023-04-27 16:18:23.623629
Iteration done at 2023-04-27 16:22:15.917784
Iteration done at 2023-04-27 16:26:05.350198
Iteration done at 2023-04-27 16:29:50.617929
Iteration done at 2023-04-27 16:33:54.046494
Iteration done at 2023-04-27 16:38:05.615128
Iteration done at 2023-04-27 16:41:46.657912
Iteration done at 2023-04-27 16:45:16.307727


KeyboardInterrupt: 

## Actual Qiskit VQE Invocation

In [ ]:
pylab.rcParams["figure.figsize"] = (12, 8)
pylab.plot(converge_counts, converge_vals, label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right")

## Calculation of reference value
The eigenvalue calculation can also be done using a simple linear algebra solver in this case.

In [ ]:
# Analytic solution
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=operator)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")